经过几年的不断优化和发展，TensorFlow 目前的代码量大约在 40 万行左右。因此，本系列课程不可能涵盖 TensorFlow 所有的内容，仅介绍其常用的操作或 API，具体如下：

 简介 - 介绍了如何使用高阶 API 之外的低阶 TensorFlow API 的基础知识。
 张量 - 介绍了如何创建、操作和访问张量（TensorFlow 中的基本对象）。
 变量 - 详细介绍了如何在程序中表示共享持久状态。
 图和会话 - 介绍了以下内容：
数据流图：这是 TensorFlow 将计算表示为操作之间的依赖关系的一种表示法。
会话：TensorFlow 跨一个或多个本地或远程设备运行数据流图的机制。如果您使用低阶 TensorFlow API 编程，请务必阅读并理解本单元的内容。如果你使用高阶 TensorFlow API（例如 Estimator 或 Keras）编程，则高阶 API 会为你创建和管理图和会话，但是理解图和会话依然对你有所帮助。
 保存和恢复 - 介绍了如何保存和恢复变量及模型。

在 TensorFlow 中，数据流图是一个 tf.Graph 对象，tf.Graph 包含两类相关信息：

图结构： 图的节点和边，表示各个操作组合在一起的方式，但不规定它们的使用方式。图结构与汇编代码类似：检查图结构可以传达一些有用的信息，但它不包含源代码传达的所有实用上下文信息。

图集合： TensorFlow 提供了一种在 tf.Graph 中存储元数据集合的通用机制。tf.add_to_collection 函数允许将对象列表与一个键关联（其中 tf.GraphKeys 定义了部分标准键），tf.get_collection 允许查询与某个键关联的所有对象。TensorFlow 库的许多部分会使用此设施资源：例如，当创建 tf.Variable 时，系统会默认将其添加到表示 “全局变量” 和 “可训练变量” 的集合中。当后续创建 tf.train.Saver 或 tf.train.Optimizer 时，这些集合中的变量将用作默认参数。

In [3]:
import tensorflow as tf


In [4]:
g_1 = tf.Graph()
with g_1.as_default():
    a = tf.constant(3.0, name='a')
    b = tf.constant(4.0, name='b')
    c = a + b

In [5]:
print(a, b, c, sep='\n')

Tensor("a:0", shape=(), dtype=float32)
Tensor("b:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [9]:
# 命名空间
e_0 = tf.constant(0, name="e")
print(e_0)
e_1 = tf.constant(2, name="e")
print(e_1)
with tf.name_scope("outer"):  # 在命名空间 outer 下创建常量
    e_2 = tf.constant(2, name="e")
    print(e_2)
    with tf.name_scope("inner"):  # 在命名空间 inter 下创建常量
        e_3 = tf.constant(3, name="e")
        print(e_3)
        e_4 = tf.constant(4, name="e")
        print(e_4)
    with tf.name_scope("inner"):
        e_5 = tf.constant(5, name="e")
e_0

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


<tf.Tensor: id=39, shape=(), dtype=int32, numpy=0>

In [10]:
a = tf.constant(3.0, name='a')  # 创建一个常量 a
b = tf.constant(4.0, name='b')  # 创建一个常量 b
c = a + b  # 创建一个操作 a+b
print(a)
print(b)
print(c)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)


# 张量
在 TensorFlow 中，张量被操作和传递的主要对象是 tf.Tensor，其具有以下属性：

数据类型（dtype）：指的是张量的数据类型，例如 float32、int32 或 string等；
形状（shape）：指的是张量的维度以及每个维度的大小，例如三维的张量：(4,2,6)。
名字（name）：指的是张量在计算图中的命名
tf.Variable 变量，其值可以在训练中被改变
tf.constant常量，其值可以在训练中不可改变
tf.placeholder占位符常量，在运行会话时，其值需要给定
tf.SparseTensor常量，稀疏张量
上面所列的四种类型的张量中，只有 tf.Variable 是可变张量，其他张量均不可改变

阶	数学实例
0	标量（只有大小）
1	矢量（大小和方向）
2	矩阵（数据表）
3	3 阶张量（数据立体）
n	n 阶张量（自行想象）

In [13]:
name = tf.Variable("Pig", tf.string)  # 0 阶张量
name

<tf.Variable 'Variable:0' shape=() dtype=string, numpy=b'Pig'>

In [14]:
name1 = tf.Variable(["Pig", "Dog"], tf.string)  # 一阶张量
name1

<tf.Variable 'Variable:0' shape=(2,) dtype=string, numpy=array([b'Pig', b'Dog'], dtype=object)>

In [17]:
name2 = tf.Variable([[3,4],[3,3]], tf.float32) # 二阶张量
name2


<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[3, 4],
       [3, 3]])>

In [18]:
my_image = tf.zeros([10, 299, 299, 3])
my_image

<tf.Tensor: id=96, shape=(10, 299, 299, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]

In [19]:
# 要确定 tf.Tensor 对象的阶，需调用 tf.rank 方法
tf.rank(my_image)

<tf.Tensor: id=98, shape=(), dtype=int32, numpy=4>

In [23]:
# 张量的切片
my_matrix = tf.constant([[4, 9], [16, 25]], tf.int32)
my_scalar = my_matrix[1, 0]
my_scalar

<tf.Tensor: id=120, shape=(), dtype=int32, numpy=16>

阶	形状	维数	示例
0	[]	0-D	0 维张量。标量。
1	[D0]	1-D	形状为 [5] 的 1 维张量。
2	[D0, D1]	2-D	形状为 [3, 4] 的 2 维张量。
3	[D0, D1, D2]	3-D	形状为 [1, 4, 3] 的 3 维张量。
n	[D0, D1, ... Dn-1]	n 维	形状为 [D0, D1, ... Dn-1] 的张量。

In [24]:
rank_three_tensor = tf.ones([3, 4, 5])  # 创建一个全为 1 的矩阵常量
print(rank_three_tensor)
matrix = tf.reshape(rank_three_tensor, [6, 10])   # 改变形状
print(matrix)
matrixB = tf.reshape(matrix, [3, -1])   # -1 表示系统自行判断大小
print(matrixB)
matrixAlt = tf.reshape(matrixB, [4, 3, -1])
print(matrixAlt)
yet_another = tf.reshape(matrixAlt, [3, 2, -1])
print(yet_another)

tf.Tensor(
[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]], shape=(3, 4, 5), dtype=float32)
tf.Tensor(
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(6, 10), dtype=float32)
tf.Tensor(
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(3, 20), dtype=float32)
tf.Tensor(
[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]], s

In [25]:
list0 = tf.constant([1, 2, 3], dtype=tf.int32)
print(list0)
float_tensor = tf.cast(list0, dtype=tf.float32)  # csat 转换数据类型
print(float_tensor)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)


# 数据集

In [28]:
my_data = [     # 初始化一个二维数组
    [0, 1, ],
    [2, 3, ],
    [4, 5, ],
    [6, 7, ],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)  # 构建数据集
# next_item = slices.make_one_shot_iterator().get_next()  # 定义一个批次取的迭代器
for i in slices:
    print(i)

tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor([2 3], shape=(2,), dtype=int32)
tf.Tensor([4 5], shape=(2,), dtype=int32)
tf.Tensor([6 7], shape=(2,), dtype=int32)


# 网络层
TensorFlow 主要是用来搭建深度学习模型的，因此其通过各种各样的  层 来创建神经网络的每一层。

层将变量和作用于它们的操作打包在一起。例如， 密集连接层 会对每个输出对应的所有输入执行加权和，并应用  激活函数 （可选）。连接权重和偏差由层对象管理。

**初始化层，执行层，层函数的快捷方式**

In [30]:
!tree

卷 新加卷 的文件夹 PATH 列表
卷序列号为 7ACF-E3F3
D:.
└─.ipynb_checkpoints
